# Subject neural tagger seq2seq with attention

This notebook trains a sequence to sequence (seq2seq) model for finding the subject (SBJ) in a sentence. This is an advanced example that assumes some knowledge of sequence to sequence models.

After training the model in this notebook, you will be able to input a Hebrew sentence, such as *"באנציקלופדיה אנחנו משתדלים לדווח כמה שפחות על דברים שקרו לאחרונה"*, and return the subject words: *"אנחנו"*

The tagging quality is reasonable for a toy example, but the generated attention plot is perhaps more interesting. This shows which parts of the input sentence has the model's attention while tagging:

for the sentence:
```"הוא לא יודע מה עובר עליי"```

![example_sentences_tagged](img/result_example.png "sentences_tagged")

Note: This example takes approximately 90 mintues to run on a single GTX 1060 GPU.

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
#!pip install tensorflow-gpu==2.0.0-alpha0
import tensorflow as tf

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import pandas as pd
import logging
import gensim
from scipy import spatial

orginal_sentences_oath_file = 'Data/Hebrew/SVLM_Hebrew_Wikipedia_Corpus.txt'
tagged_sentences_path_file = 'Data/Hebrew/parsed.txt'
csv_hebrew_sentences_path_file = 'Data/Hebrew/Hebrew_tagged_sentences.csv'
NUM_EXAMPLES = 50000
EMBEDDING_DIM = 300

## Tagging and Preprocessing the dataset

We'll use a [curpus](https://github.com/NLPH/SVLM-Hebrew-Wikipedia-Corpus/blob/master/SVLM_Hebrew_Wikipedia_Corpus.txt) built by Dr. Vered Silber-Varod and Prof. Ami Moyal as part of their work on [paper](https://github.com/NLPH/SVLM-Hebrew-Wikipedia-Corpus/blob/master/Phonemes_freqency_Silber-Varod-Latin-Moyal.pdf).

The SVLM Hebrew Wikipedia Courpus is a corpus made up of 50,000 Hebrew sentences from the Hebrew Wikipedia chosen to ensure phoneme coverage for the purpose of a sentence recording project

This Corpus contains sentence in the format:
```
פרסומים עיקריים מקורות המחשבה הצבאית המודרנית משרד הביטחון ההוצאה לאור
קישורים חיצוניים אתר האינטרנט של המרכז הירושלמי לענייני ציבור ומדינה
```
As it was generated from Hebrew Wikipedia sources, which are licensed under the [CC-BY-SA 3.0](https://creativecommons.org/licenses/by-sa/3.0/) license, this corpus is thus also necessarilly licensed under the same license. 

### tagging

To tag the sentences we'll use Hebrew Dependency Parser  [Yoav Goldberg, September 2011](https://www.cs.bgu.ac.il/~yoavg/software/hebparsers/hebdepparser/).

For example to the input:
```
קורות חייו צמרת למד בחוגים לחינוך ופילוסופיה באוניברסיטה העברית בירושלים
```
the parser will output parsing sentence like this (it will be in the location - 'Data/Hebrew/parsed.txt' ) : 

![sentences_tagged](img/sentences_tagged.png "sentences_tagged")

we will take only the subject word (SBJ):![sentences_tagged_SBJ](img/sentences_tagged_sbj.png "sentences_tagged")

### Preprocessing

Here are the steps we'll take to prepare the data:

1. extract subject for each tagged sentence.
2. flat the list of subject to one string.
3. match subject to orginal sentence.
4. make csv.

In [2]:
#extract subject for each tagged sentence
def extract_sbj(sentences):
    listoflists = []
    sublist = []
    for i in sentences:
        if i.find("SBJ") !=-1:
            i = re.sub(r"[^א-ת\"]+", " ", i)
            if len(i)>3:
                sublist.append(i)
        elif i == '\n':
            listoflists.append(sublist)
            sublist = []
    return listoflists

#flat the list of subject to one string
def listToString(s):  
    str1 = " " 
    return (str1.join(s))

#match subject to orginal sentence
def make_pairs(orginal_sentences,tagged_sentences,num_examples):
    subjects = extract_sbj(tagged_sentences)
    clear_lines = []
    for tag, sentence in zip(subjects[:num_examples],orginal_sentences[:num_examples]):
        clear_lines.append(listToString(tag) + '\t' + sentence)
    subject = []
    sentences = []
    for line in clear_lines:
        line = line.split('\t')
        sentences.append(line[1])
        subject.append(line[0])
    return subject, sentences

def make_csv_subject_sentence(orginal_sentences,tagged_sentences, csv_path,num_examples):
    subjects, sentences = make_pairs(orginal_sentences,tagged_sentences,num_examples)
    trainDF = pd.DataFrame()
    trainDF['subject'] = subjects[:num_examples]
    trainDF['sentence'] = sentences[:num_examples]
    mask = (trainDF['subject'].str.len()>1)
    trainDF = trainDF.loc[mask]
    trainDF.to_csv(csv_path)

In [3]:
orginal_sentences = open(orginal_sentences_oath_file).readlines()
tagged_sentences = open(tagged_sentences_path_file).readlines()
make_csv_subject_sentence(orginal_sentences,tagged_sentences,csv_hebrew_sentences_path_file,NUM_EXAMPLES)

In [4]:
df = pd.read_csv(csv_hebrew_sentences_path_file,index_col=[0])
NUM_EXAMPLES = df.shape[0]
df

,subject,sentence
0,קורות,קורות חייו צמרת למד בחוגים לחינוך ופילוסופיה ב...
3,פרסומים,פרסומים עיקריים מקורות המחשבה הצבאית המודרנית ...
7,מרוקו,מרוקו הצטרפה למשפחת האירוויזיון בפעם הראשונה ו...
12,קו,כתוצאה ממלחמת העצמאות ובעקבות הסכמי שביתת הנשק...
13,הסבר,הסבר לחשיבות הקריטריון ניתן למצוא בדיון שנערך ...
...,...,...
49993,זה ערך,אבל זה לא הערך הערך הוא הרב טאו\n
49995,הוא,ב כל מהנדס חומרה הוא מהנדס חשמל\n
49996,דרך,דרך בורמה בורמה היא הדרך המקורית\n
49998,היסטוריה,גם היסטוריה רוחנית גם היסטוריה מדינית\n


## Prepare the dataset

The csv contains sentences & subjects pairs in the format:

In [5]:
df.head()

,subject,sentence
0,קורות,קורות חייו צמרת למד בחוגים לחינוך ופילוסופיה ב...
3,פרסומים,פרסומים עיקריים מקורות המחשבה הצבאית המודרנית ...
7,מרוקו,מרוקו הצטרפה למשפחת האירוויזיון בפעם הראשונה ו...
12,קו,כתוצאה ממלחמת העצמאות ובעקבות הסכמי שביתת הנשק...
13,הסבר,הסבר לחשיבות הקריטריון ניתן למצוא בדיון שנערך ...


### preparing & Cleaning:

Here are the steps we'll take to prepare the data:

1. Add a *start* and *end* token to each sentence.
2. Clean the sentences by removing special characters.
3. Create a word index and reverse word index (dictionaries mapping from word → id and id → word).
4. Pad each sentence to a maximum length.

In [6]:
se11 = orginal_sentences[1] + "\"" +" dfsfdd"+"'"
k='\"'
sell1 = se11.replace(k,'')

In [7]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence_Hebrew(w):
    #w = unicode_to_ascii(w.lower().strip())
    wn = [k.replace('\"'+'\'','') for k in w.split()]
    w = listToString(wn) 
    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    #w = re.sub(r"([?.!,¿\"])", r" \1 ", w)
    #w = re.sub(r'[" "]+', " ", w)
    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    k='\"'
    w = w.replace(k,'')
    w = re.sub(r"[^א-ת]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [8]:
# Converts the unicode file to ascii
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn')


def preprocess_sentence_English(w):
    w = unicode_to_ascii(w.lower().strip())

    # creating a space between a word and the punctuation following it
    # eg: "he is a boy." => "he is a boy ."
    # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
    w = re.sub(r"([?.!,¿])", r" \1 ", w)
    w = re.sub(r'[" "]+', " ", w)

    # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)

    w = w.rstrip().strip()

    # adding a start and an end token to the sentence
    # so that the model know when to start and stop predicting.
    w = '<start> ' + w + ' <end>'
    return w

In [9]:
example_sentence = preprocess_sentence_Hebrew("מבחינתי אפשר להעלות אותו לרשימת ההמתנה לשמוע הערות ואח\"כ להצבעה")
print(example_sentence)
example_sentence = [subject for subject in example_sentence.split()]

<start> מבחינתי אפשר להעלות אותו לרשימת ההמתנה לשמוע הערות ואחכ להצבעה <end>


In [10]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [11]:
def create_dataset_from_csv(csv_path, num_examples):
    df = pd.read_csv(csv_path,index_col=[0])
    subjects = [preprocess_sentence_Hebrew(subject) for subject in df['subject']]  
    sentences = [preprocess_sentence_Hebrew(sentence) for sentence in df['sentence']] 
    subjects = [subject.split() for subject in subjects]
    sentences = [sentence.split() for sentence in sentences]
    return subjects, sentences,df

In [12]:
subjects, sentences,df = create_dataset_from_csv(csv_hebrew_sentences_path_file, NUM_EXAMPLES)

## Cross-validation: evaluating estimator performance

Learning the parameters of a prediction function and testing it on the same data is a methodological mistake: a model that would just repeat the labels of the samples that it has just seen would have a perfect score but would fail to predict anything useful on yet-unseen data. This situation is called **overfitting**. To avoid it, it is common practice when performing a (supervised) machine learning experiment to hold out part of the available data as a test set `x_test, y_test`. Note that the word “experiment” is not intended to denote academic use only, because even in commercial settings machine learning usually starts out experimentally. Here is a flowchart of typical cross validation workflow in model training. The best parameters can be determined by grid search techniques.

<img src="https://scikit-learn.org/stable/_images/grid_search_workflow.png" alt="Drawing" style="width: 500px;"/>

*Grid Search Workflow*

In scikit-learn a random split into training and test sets can be quickly computed with the **`train_test_split`** helper function.

### Parameters

#### arrays:
sequence of indexables with same length / shape[0]
Allowed inputs are lists, numpy arrays, scipy-sparse matrices or pandas dataframes.

#### test_size:
If float, should be between 0.0 and 1.0 and represent the proportion of the dataset to include in the test split. If int, represents the absolute number of test samples. If None, the value is set to the complement of the train size. If train_size is also None, it will be set to 0.25.

Let’s load the tagged sentencses data:

In [13]:
sentences_train, sentences_test, subjects_train, subjects_test =\
train_test_split(sentences, subjects, test_size = 0.25)

## Training the Word2Vec model

Training the model is fairly straightforward. You just instantiate Word2Vec and pass the reviews that we read in the previous step. So, we are essentially passing on a list of lists. Where each list within the main list contains a set of tokens from a user review. Word2Vec uses all these tokens to internally create a vocabulary. And by vocabulary, I mean a set of unique words.

### Understanding some of the parameters
To train the model earlier, we had to set some parameters. Now, let's try to understand what some of them mean. For reference, this is the command that we used to train the model.

`word_model = gensim.models.Word2Vec(subjects+sentences, size=EMBEDDING_DIM, min_count=1, window=5, iter=100)`

#### size
The size of the dense vector to represent each token or word. If you have very limited data, then size should be a much smaller value. If you have lots of data, its good to experiment with various sizes. A value of 100-150 has worked well for me.

#### window
The maximum distance between the target word and its neighboring word. If your neighbor's position is greater than the maximum window width to the left and the right, then, some neighbors are not considered as being related to the target word. In theory, a smaller window should give you terms that are more related. If you 
have lots of data, then the window size should not matter too much, as long as its a decent sized window.

<img src="https://miro.medium.com/max/1600/0*1uA0SYcKU_dLTj-V.png" alt="Drawing" style="width: 500px;"/>
<center>C is the window size</center>


#### min_count
Minimium frequency count of words. The model would ignore words that do not statisfy the min_count. Extremely infrequent words are usually unimportant, so its best to get rid of those. Unless your dataset is really tiny, this does not really affect the model.

#### workers
How many threads to use behind the scenes?

#### iter
Number of iterations (epochs) over the corpus.

### When should you use Word2Vec?
There are many application scenarios for Word2Vec. Imagine if you need to build a sentiment lexicon. Training a Word2Vec model on large amounts of user reviews helps you achieve that. You have a lexicon for not just sentiment, but for most words in the vocabulary.

Beyond, raw unstructured text data, you could also use Word2Vec for more structured data. For example, if you had tags for a million stackoverflow questions and answers, you could find tags that are related to a given tag and recommend the related ones for exploration. You can do this by treating each set of co-occuring tags as a "sentence" and train a Word2Vec model on this data. Granted, you still need a large number of examples to make it work.

In [14]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
word_model = gensim.models.Word2Vec(subjects+sentences, size=EMBEDDING_DIM, min_count=1, window=5, iter=100)

In [15]:
#Each word in the dictionary has an index
def word2idx(word, word_model):
    if word in word_model.wv.vocab:
        return word_model.wv.vocab[word].index
    else:
        return 1 #default index for non-exsits in vec_model voacb
    
def idx2word(idx, word_model):
  return word_model.wv.index2word[idx]

In [16]:
def convert(sentence, word_model):
  for w in sentence:
      print ("%d ----> %s" % (word2idx(w,word_model),w))

In [17]:
#padding the sentences with <end> tag index
def sentence_to_indexes(sentence,word_model,max_length_inp):
    data_set = [word2idx(w,word_model) for w in sentence]
    data_set = tf.keras.preprocessing.sequence.pad_sequences([data_set],max_length_inp, padding='post', value=1)
    return data_set

In [18]:
#create matrix of the dataset indexes 
def create_dataset_word2vec_matrix(sentences,vec_model,max_length_inp):
    data_set = []
    #creates train data set
    for i, sentence in enumerate(sentences):
        data_set.append(list())
        data_set[i] = ([word2idx(word,vec_model) for word in sentence])
    data_set = tf.keras.preprocessing.sequence.pad_sequences(data_set,max_length_inp, padding='post', value=1)
    return data_set 

In [19]:
max_sentence_len, max_subjects_len  = max_length(sentences_train), max_length(subjects_train)
x_train = create_dataset_word2vec_matrix(sentences_train,word_model,max_sentence_len)
y_train = create_dataset_word2vec_matrix(subjects_train,word_model,max_subjects_len)
x_test = sentences_test
y_test = subjects_test
max_length_inp, max_length_targ  = x_train.shape[1], y_train.shape[1]

In [20]:
#example sentence
convert(example_sentence,word_model)
sentence_to_indexes(example_sentence,word_model,max_length_inp)

0 ----> <start>
232 ----> מבחינתי
78 ----> אפשר
820 ----> להעלות
48 ----> אותו
4238 ----> לרשימת
10520 ----> ההמתנה
1129 ----> לשמוע
402 ----> הערות
14243 ----> ואחכ
1618 ----> להצבעה
1 ----> <end>


array([[    0,   232,    78,   820,    48,  4238, 10520,  1129,   402,
        14243,  1618,     1,     1,     1]], dtype=int32)

In [21]:
#input matrix
x_train, x_train.shape

(array([[    0, 15308,  4270, ...,     1,     1,     1],
        [    0,   213,     9, ...,     1,     1,     1],
        [    0,     2,   235, ...,     1,     1,     1],
        ...,
        [    0,   544,  1376, ...,     1,     1,     1],
        [    0,   239,  1376, ...,     1,     1,     1],
        [    0,  9214,     6, ...,     1,     1,     1]], dtype=int32),
 (26876, 14))

In [22]:
BUFFER_SIZE = len(x_train)
BATCH_SIZE = 64
steps_per_epoch = len(x_train)//BATCH_SIZE
embedding_dim = EMBEDDING_DIM
units = 1024
pretrained_weights = word_model.wv.syn0
vocab_size = word_model.wv.syn0.shape[0]
emdedding_size = word_model.wv.syn0.shape[1]

/home/mishka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  
/home/mishka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  import sys
/home/mishka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.vectors instead).
  


In [23]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).shuffle(len(y_train))
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [24]:
example_input_batch, _ = next(iter(dataset))
example_input_batch.shape

TensorShape([64, 14])

## Write the encoder and decoder model

Implement an encoder-decoder model with attention which you can read about in the TensorFlow [Neural Machine Translation (seq2seq) tutorial](https://github.com/tensorflow/nmt). This example uses a more recent set of APIs. This notebook implements the [attention equations](https://github.com/tensorflow/nmt#background-on-the-attention-mechanism) from the seq2seq tutorial. The following diagram shows that each input words is assigned a weight by the attention mechanism which is then used by the decoder to predict the next word in the sentence. The below picture and formulas are an example of attention mechanism from [Luong's paper](https://arxiv.org/abs/1508.04025v5). 

<img src="https://www.tensorflow.org/images/seq2seq/attention_mechanism.jpg" width="500" alt="attention mechanism">

The input is put through an encoder model which gives us the encoder output of shape *(batch_size, max_length, hidden_size)* and the encoder hidden state of shape *(batch_size, hidden_size)*.

Here are the equations that are implemented:

<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_0.jpg" alt="attention equation 0" width="800">
<img src="https://www.tensorflow.org/images/seq2seq/attention_equation_1.jpg" alt="attention equation 1" width="800">

This tutorial uses [Bahdanau attention](https://arxiv.org/pdf/1409.0473.pdf) for the encoder. Let's decide on notation before writing the simplified form:

* FC = Fully connected (dense) layer
* EO = Encoder output
* H = hidden state
* X = input to the decoder

And the pseudo-code:

* `score = FC(tanh(FC(EO) + FC(H)))`
* `attention weights = softmax(score, axis = 1)`. Softmax by default is applied on the last axis but here we want to apply it on the *1st axis*, since the shape of score is *(batch_size, max_length, hidden_size)*. `Max_length` is the length of our input. Since we are trying to assign a weight to each input, softmax should be applied on that axis.
* `context vector = sum(attention weights * EO, axis = 1)`. Same reason as above for choosing axis as 1.
* `embedding output` = The input to the decoder X is passed through an embedding layer.
* `merged vector = concat(embedding output, context vector)`
* This merged vector is then given to the LSTM

The shapes of all the vectors at each step have been specified in the comments in the code:

In [25]:
class Encoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim,pretrained_weights, maxlen, enc_units, batch_sz):
    super(Encoder, self).__init__()
    self.batch_sz = batch_sz
    self.enc_units = enc_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,weights=[pretrained_weights],input_length=maxlen,trainable=True)
    self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(self.enc_units, return_sequences=True, return_state=True))

  def call(self, x, forward_h, forward_c, backward_h, backward_c):
    x = self.embedding(x)
    output, st_forward_h, st_forward_c, st_backward_h, st_backward_c = self.lstm(x, initial_state = [forward_h, forward_c, backward_h, backward_c])
    return output, st_forward_h, st_forward_c, st_backward_h, st_backward_c

  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units)), tf.zeros((self.batch_sz, self.enc_units))

In [26]:
encoder = Encoder(vocab_size, emdedding_size, pretrained_weights, max_length_inp ,units, BATCH_SIZE)
forward_h, forward_c, backward_h, backward_c = encoder.initialize_hidden_state()
sample_output, forward_h, forward_c, backward_h, backward_c = encoder(example_input_batch, forward_h, forward_c, backward_h, backward_c)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden forward_h state shape: (batch size, units) {}'.format(forward_h.shape))
print ('Encoder Hidden forward_c state shape: (batch size, units) {}'.format(forward_c.shape))
print ('Encoder Hidden backward_h state shape: (batch size, units) {}'.format(backward_h.shape))
print ('Encoder Hidden backward_c state shape: (batch size, units) {}'.format(backward_c.shape))

Encoder output shape: (batch size, sequence length, units) (64, 14, 2048)
Encoder Hidden forward_h state shape: (batch size, units) (64, 1024)
Encoder Hidden forward_c state shape: (batch size, units) (64, 1024)
Encoder Hidden backward_h state shape: (batch size, units) (64, 1024)
Encoder Hidden backward_c state shape: (batch size, units) (64, 1024)


In [27]:
class BahdanauAttention(tf.keras.Model):
  def __init__(self, units):
    super(BahdanauAttention, self).__init__()
    self.W1 =  tf.keras.layers.Dense(units)
    self.W2 =  tf.keras.layers.Dense(units)
    self.V =  tf.keras.layers.Dense(1)

  def call(self, forward_h, forward_c, backward_h, backward_c, values):
    hidden_h =  tf.keras.layers.Concatenate()([forward_h, backward_h])
    hidden_c =  tf.keras.layers.Concatenate()([forward_c, backward_c])
    query =  tf.keras.layers.Concatenate()([hidden_h, hidden_c])

    hidden_with_time_axis = tf.expand_dims(query, 1)

    score = self.V(tf.nn.tanh(
        self.W1(values) + self.W2(hidden_with_time_axis)))

    attention_weights = tf.nn.softmax(score, axis=1)

    context_vector = attention_weights * values

    context_vector = tf.reduce_sum(context_vector, axis=1)
    
    return context_vector, attention_weights


In [28]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, pretrained_weights, maxlen, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim,weights=[pretrained_weights],input_length=maxlen,trainable=True)
    self.lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(self.dec_units, return_sequences=True, return_state=True))
    self.fc = tf.keras.layers.Dense(vocab_size)

    # used for attention
    self.attention = BahdanauAttention(self.dec_units)

  def call(self, x, forward_h, forward_c, backward_h, backward_c, enc_output):
    # enc_output shape == (batch_size, max_length, hidden_size)
    context_vector, attention_weights = self.attention(forward_h, forward_c, backward_h, backward_c, enc_output)

    # x shape after passing through embedding == (batch_size, 1, embedding_dim)
    x = self.embedding(x)

    # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
    x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

    # passing the concatenated vector to the bi-LSTM
    output, st_forward_h, st_forward_c, st_backward_h, st_backward_c = self.lstm(x)

    # output shape == (batch_size * 1, hidden_size)
    output = tf.reshape(output, (-1, output.shape[2]))

    # output shape == (batch_size, vocab)
    x = self.fc(output)
    return x, st_forward_h, st_forward_c, st_backward_h, st_backward_c, attention_weights


In [29]:
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(forward_h, forward_c, backward_h, backward_c, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

decoder = Decoder(vocab_size, emdedding_size, pretrained_weights, max_length_targ ,units, BATCH_SIZE)

sample_decoder_output, _, _, _, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      forward_h, forward_c, backward_h, backward_c, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Attention result shape: (batch size, units) (64, 2048)
Attention weights shape: (batch_size, sequence_length, 1) (64, 14, 1)
Decoder output shape: (batch_size, vocab size) (64, 20947)


## Define the optimizer and the loss function

In [30]:
optimizer = tf.keras.optimizers.Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

In [31]:
def loss_function(real, pred):
  mask = tf.math.logical_not(tf.math.equal(real, 0))
  loss_ = loss_object(real, pred)

  mask = tf.cast(mask, dtype=loss_.dtype)
  loss_ *= mask

  return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [32]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [33]:
@tf.function
def train_step(inp, targ, forward_h, forward_c, backward_h, backward_c):
  loss = 0

  with tf.GradientTape() as tape:
    enc_output, enc_forward_h, enc_forward_c, enc_backward_h,\
                    enc_backward_c = encoder(inp,forward_h, forward_c, backward_h, backward_c)

    dec_forward_h, dec_forward_c, dec_backward_h, dec_backward_c = \
    enc_forward_h, enc_forward_c, enc_backward_h, enc_backward_c

    dec_input = tf.expand_dims([word_model.wv.vocab['<start>'].index] * BATCH_SIZE, 1)

    # Teacher forcing - feeding the target as the next input
    for t in range(1, targ.shape[1]):
      # passing enc_output to the decoder
      predictions, dec_forward_h, dec_forward_c, dec_backward_h,dec_backward_c, _ =\
                decoder(dec_input, dec_forward_h, dec_forward_c, dec_backward_h, dec_backward_c, enc_output)

      loss += loss_function(targ[:, t], predictions)

      # using teacher forcing
      dec_input = tf.expand_dims(targ[:, t], 1)

  batch_loss = (loss / int(targ.shape[1]))

  variables = encoder.trainable_variables + decoder.trainable_variables

  gradients = tape.gradient(loss, variables)

  optimizer.apply_gradients(zip(gradients, variables))

  return batch_loss

In [34]:
EPOCHS = 100
first = time.time()
for epoch in range(EPOCHS):
  start = time.time()

  enc_forward_h, enc_forward_c, enc_backward_h, enc_backward_c = encoder.initialize_hidden_state()
  total_loss = 0

  for (batch, (inp, targ)) in enumerate(dataset.take(steps_per_epoch)):
    batch_loss = train_step(inp, targ, enc_forward_h, enc_forward_c, enc_backward_h, enc_backward_c)
    total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                     batch,
                                                     batch_loss.numpy()))
  # saving (checkpoint) the model every 2 epochs
  if (epoch + 1) % 2 == 0:
    checkpoint.save(file_prefix = checkpoint_prefix)

  print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
  print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))
    
print('Time taken for 1 epoch {} sec\n'.format(time.time() - first))

Epoch 1 Batch 0 Loss 8.5308
Epoch 1 Batch 100 Loss 1.2316
Epoch 1 Batch 200 Loss 1.2348
Epoch 1 Batch 300 Loss 1.2670
Epoch 1 Batch 400 Loss 1.2638
Epoch 1 Loss 1.3801
Time taken for 1 epoch 180.82781744003296 sec

Epoch 2 Batch 0 Loss 1.3274
Epoch 2 Batch 100 Loss 0.9768
Epoch 2 Batch 200 Loss 1.0512
Epoch 2 Batch 300 Loss 1.0432
Epoch 2 Batch 400 Loss 1.0754
Epoch 2 Loss 1.0566
Time taken for 1 epoch 157.61174082756042 sec

Epoch 3 Batch 0 Loss 1.1174
Epoch 3 Batch 100 Loss 0.7723
Epoch 3 Batch 200 Loss 0.9082
Epoch 3 Batch 300 Loss 0.8143
Epoch 3 Batch 400 Loss 0.8558
Epoch 3 Loss 0.8751
Time taken for 1 epoch 153.41838026046753 sec

Epoch 4 Batch 0 Loss 0.8985
Epoch 4 Batch 100 Loss 0.5753
Epoch 4 Batch 200 Loss 0.6713
Epoch 4 Batch 300 Loss 0.5851
Epoch 4 Batch 400 Loss 0.5691
Epoch 4 Loss 0.6493
Time taken for 1 epoch 156.80947184562683 sec

Epoch 5 Batch 0 Loss 0.6153
Epoch 5 Batch 100 Loss 0.3918
Epoch 5 Batch 200 Loss 0.4023
Epoch 5 Batch 300 Loss 0.3767
Epoch 5 Batch 400 Loss

Epoch 38 Batch 300 Loss 0.0004
Epoch 38 Batch 400 Loss 0.0001
Epoch 38 Loss 0.0019
Time taken for 1 epoch 157.52376651763916 sec

Epoch 39 Batch 0 Loss 0.0002
Epoch 39 Batch 100 Loss 0.0009
Epoch 39 Batch 200 Loss 0.0003
Epoch 39 Batch 300 Loss 0.0013
Epoch 39 Batch 400 Loss 0.0017
Epoch 39 Loss 0.0023
Time taken for 1 epoch 153.82075381278992 sec

Epoch 40 Batch 0 Loss 0.0008
Epoch 40 Batch 100 Loss 0.0031
Epoch 40 Batch 200 Loss 0.0294
Epoch 40 Batch 300 Loss 0.0159
Epoch 40 Batch 400 Loss 0.0447
Epoch 40 Loss 0.0065
Time taken for 1 epoch 157.84073066711426 sec

Epoch 41 Batch 0 Loss 0.0022
Epoch 41 Batch 100 Loss 0.0013
Epoch 41 Batch 200 Loss 0.0061
Epoch 41 Batch 300 Loss 0.0015
Epoch 41 Batch 400 Loss 0.0014
Epoch 41 Loss 0.0072
Time taken for 1 epoch 152.91706800460815 sec

Epoch 42 Batch 0 Loss 0.0102
Epoch 42 Batch 100 Loss 0.0015
Epoch 42 Batch 200 Loss 0.0063
Epoch 42 Batch 300 Loss 0.0035
Epoch 42 Batch 400 Loss 0.0010
Epoch 42 Loss 0.0045
Time taken for 1 epoch 156.652067

Epoch 75 Batch 400 Loss 0.0006
Epoch 75 Loss 0.0022
Time taken for 1 epoch 153.08627700805664 sec

Epoch 76 Batch 0 Loss 0.0028
Epoch 76 Batch 100 Loss 0.0003
Epoch 76 Batch 200 Loss 0.0010
Epoch 76 Batch 300 Loss 0.0001
Epoch 76 Batch 400 Loss 0.0004
Epoch 76 Loss 0.0015
Time taken for 1 epoch 155.06479859352112 sec

Epoch 77 Batch 0 Loss 0.0008
Epoch 77 Batch 100 Loss 0.0001
Epoch 77 Batch 200 Loss 0.0001
Epoch 77 Batch 300 Loss 0.0007
Epoch 77 Batch 400 Loss 0.0001
Epoch 77 Loss 0.0013
Time taken for 1 epoch 153.02718591690063 sec

Epoch 78 Batch 0 Loss 0.0000
Epoch 78 Batch 100 Loss 0.0002
Epoch 78 Batch 200 Loss 0.0002
Epoch 78 Batch 300 Loss 0.0043
Epoch 78 Batch 400 Loss 0.0000
Epoch 78 Loss 0.0007
Time taken for 1 epoch 154.55366563796997 sec

Epoch 79 Batch 0 Loss 0.0000
Epoch 79 Batch 100 Loss 0.0000
Epoch 79 Batch 200 Loss 0.0000
Epoch 79 Batch 300 Loss 0.0001
Epoch 79 Batch 400 Loss 0.0000
Epoch 79 Loss 0.0006
Time taken for 1 epoch 152.7528874874115 sec

Epoch 80 Batch 0 L

In [50]:
def evaluate(sentence):
    attention_plot = np.zeros((max_length_targ, max_length_inp))

    inputs = sentence_to_indexes(sentence,word_model, max_length_inp)
    inputs = tf.convert_to_tensor(inputs)

    result = ''
    forward_h  = tf.zeros((1, units))
    forward_c  = tf.zeros((1, units))
    backward_h = tf.zeros((1, units))
    backward_c = tf.zeros((1, units))

    enc_out, enc_forward_h, enc_forward_c, enc_backward_h, enc_backward_c = encoder(inputs, forward_h, forward_c, backward_h, backward_c)

    dec_forward_h, dec_forward_c, dec_backward_h, dec_backward_c = enc_forward_h, enc_forward_c, enc_backward_h, enc_backward_c


    dec_input = tf.expand_dims([word_model.wv.vocab['<start>'].index], 0)

    for t in range(max_length_targ):
        predictions, dec_forward_h, dec_forward_c, dec_backward_h, dec_backward_c, attention_weights = decoder(dec_input,
                                                                                                               dec_forward_h, dec_forward_c, dec_backward_h, dec_backward_c,
                                                                                                               enc_out)

        # storing the attention weights to plot later on
        attention_weights = tf.reshape(attention_weights, (-1, ))
        attention_plot[t] = attention_weights.numpy()
        
        predicted_id = tf.argmax(predictions[0]).numpy()
        
        if word_model.wv.index2word[predicted_id] != '<end>':
            result += word_model.wv.index2word[predicted_id] + ' '

        if word_model.wv.index2word[predicted_id] == '<end>':
            return result, sentence, attention_plot

        # the predicted ID is fed back into the model
        dec_input = tf.expand_dims([predicted_id], 1)

    return result, sentence, attention_plot

In [51]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1)
    ax.matshow(attention, cmap='viridis')

    fontdict = {'fontsize': 14}

    ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
    ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

    plt.show()

## KDTree - nearest-neighbor

The algorithm used is described in Maneewongvatana and Mount 1999. The general idea is that the kd-tree is a binary tree, each of whose nodes represents an axis-aligned hyperrectangle. Each node specifies an axis and splits the set of points based on whether their coordinate along that axis is greater than or less than a particular value.

During construction, the axis and splitting point are chosen by the “sliding midpoint” rule, which ensures that the cells do not all become long and thin.

The tree can be queried for the r closest neighbors of any given point (optionally returning only those within some maximum distance of the point). It can also be queried, with a substantial gain in efficiency, for the r approximate closest neighbors.

For large dimensions (20 is already large) do not expect this to run significantly faster than brute force. High-dimensional nearest-neighbor queries are a substantial open problem in computer science.

The tree also supports all-neighbors queries, both with arrays of points and with other kd-trees. These do use a reasonably efficient algorithm, but the kd-tree is not necessarily the best data structure for this sort of calculation.

![kdtree](img/kdtree-ops.png "sentences_tagged")

`class scipy.spatial.KDTree(data, leafsize=10)[source]`

kd-tree for quick nearest-neighbor lookup

This class provides an index into a set of k-dimensional points which can be used to rapidly look up the nearest neighbors of any point.

In [52]:
def KDtree_nearest(test_sentence,result):
    A = [word_model[word] for word in test_sentence.split()]
    tree = spatial.KDTree(A)
    if(len(result)>1):
        word_result = result.split()
    else:word_result=result
    a = word_model[word_result[0].rstrip().strip()]
    index = tree.query(a)[1]
    return test_sentence.split()[index]

## Tag

* The evaluate function is similar to the training loop, except we don't use *teacher forcing* here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.
* Stop predicting when the model predicts the *end token*.
* And store the *attention weights for every time step*.
* Run **KDTree** to find the right word from the sentence*.

Note: The encoder output is calculated only once for one input.

In [53]:
def tag(sentence):
    result, sentence, attention_plot = evaluate(sentence)
    result = KDtree_nearest(sentence,result)
    print('Input: %s' % (sentence))
    print('Predicted subject: {}'.format(result))
    sentence = sentence[::-1]
    result = result[::-1]
    sentence_splited = sentence.split(' ')
    #to fix the hebrew changes
    sentence_splited.reverse()
    attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
    plot_attention(attention_plot, sentence_splited, result.split(' '))
    return result

## restoring the latest checkpoint in checkpoint_dir

In [54]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [55]:
# test_sentence = "הוא לא יודע מה עובר עליי"
# result = tag(test_sentence)

In [56]:
def listToString(s):  
    str1 = " " 
    return (str1.join(s))

In [57]:
len(sentences_test)

8959

In [66]:
new_sentences_test = [listToString(sen[1:-1]) for sen in sentences_test]
new_subjects_test = [listToString(sen[1:-1]) for sen in subjects_test]

In [67]:
def new_tag(sentence):
    result, sentence, attention_plot = evaluate(sentence)
    if len(result)>0:
        result = KDtree_nearest(sentence,result)
    return result

In [68]:
def dist(word1,word2,sen):
    x = 0
    y = 0
    for i in sen.split():
        if i.find(word1)!=-1:
            x = sen.split().index(i)
        if i.find(word2)!=-1:
            y = sen.split().index(i)
    print(x,y)
    print(abs(x-y))
    return 1-(abs(x-y)/(len(sen.split())-1))

In [69]:
dist("im","go","im go to sleep")

0 1
1


0.6666666666666667

In [70]:
new_sentences_test[14]

'מעבר לזה אני מצטרף לדברי עוזי'

In [73]:
from tqdm.notebook import trange, tqdm
# new_sentences_test = new_sentences_test[:1000]
counter = 0
real_count = 0
sentence_counter = 0
new_counter = 0
for index,sen1 in enumerate(tqdm(new_sentences_test)):
    if len(new_subjects_test[index]) != 0 and len(new_tag(sen1)) != 0:
            sentence_counter = sentence_counter+1
            for idx,word in enumerate(new_tag(sen1).split()):
#                 new_counter = new_counter + dist(word,new_subjects_test[index].split()[idx],sen1)
#                 print(dist(word,new_subjects_test[index].split()[idx],sen1))
                print(sen1)
                print(word,new_subjects_test[index].split()[idx])
                if(new_subjects_test[index].find(word)!=-1 or word.find(new_subjects_test[index])!=-1):real_count = real_count + 1;print(1)
#                 counter = counter + word_model.similarity(word,new_subjects_test[index].split()[idx])

/home/mishka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/home/mishka/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  import sys


מאידך ויקישיתוף הוא אוסף הצילומים של קרן ויקימדיה
הצילומים ויקישיתוף
בתור כותב אתה צריך לדעת מה חשוב ומה חשוב פחות
אתה מה
הוא מזוהה עם קהילת יהודי סלוניקי בעיר
סלוניקי הוא
בדרום הודו נפוץ ספורט דומה
הודו ספורט
מנועי החיפוש צריכים להתגבר על שגיאות
מנועי מנועי
1
אבל היא תמחק וחבל לי על העבודה
וחבל היא
ג וליה הייתה הבכורה בין שלושת ילדי המשפחה
ילדי ג
גורם נוסף לייצור מיותר של גז הינו לחץ פסיכולוגי
גז גורם
מנהל המרכז הנוכחי הוא דר צבי צמרת
צמרת מנהל
לגבי דבריך אני מסכים חלקית
חלקית אני
יובל מדר יובל ראה תשובתי בדף השיחה שלך
מדר יובל
אני חושב למשל על מכלול הידע המדעי כעל דלי ענק
דלי אני
להיות חופשי פירושו אנרגיית קשר אפס לפחות
אנרגיית פירושו
לכן הצלב ההפוך כיום נקרא צלב פטרוס
ההפוך צלב
מעבר לזה אני מצטרף לדברי עוזי
מצטרף אני
כל התשובות לגיטימיות רק צריכות להיות ברורות
לגיטימיות תשובות
אין רע רסג תקף את העניין שיש רע
רסג רסג
1
בשלב ראשון קובעים משפחה של מבחנים
מבחנים משפחה
קישורים חיצוניים פרופיל מאמן באתר הרשמי של ליברפול
מאמן קישורים
עוד בחייו הוא נתן כספים רבים למטרות שונות
הוא הוא
1
השעו

מועצת המערכת כוללת עשרים עורכי נושאים וחמישה עורכים ראשיים
ראשיים מועצת
העיצוב הפנימי של הארמון כמעט ולא השתנה מאז בנייתו
הפנימי עיצוב
בתקופת המאורעות הותקפה השכונה בידי הכפרים השכנים
הותקפה שכונה
למעשה רב הערכים שכתבתי היו בהיסטוריה והיסטוריה צבאית
והיסטוריה רב
לשם כך הקימה האקדמיה את הזרוע למחקר בסיסי
הזרוע אקדמיה
ככל הידוע ג ונסון היה נשוי פעמיים
ג ידוע
יש כבר את הערך אבודים דמויות והוא מספיק לחלוטין
אבודים את
קורות חיים פרק זה לוקה בחסר
זה קורות
אני מניח שזה עניין של חירות המשורר
חירות אני
אין קשר פנימי בין צירוף האותיות שולחן לבין השולחן כשלעצמו
כשלעצמו קשר
היסטוריונים מעריכים את אוכלוסיית העיר באותה העת כעשרים אלף נפש
מעריכים היסטוריונים
דף השינויים האחרונים לא שווה ויקיפדים כמוך
השינויים דף
אז בלי היערות כי אין לזה סוף
היערות סוף
הייתי רוצה שמרבית השאלות יהיו פתוחות ולא מתן ציונים
שמרבית מרבית
1
המצב הנוכחי של קרתגו לעומת חורבות קרתגו איננו נסבל
חורבות מצב
הגעתי לזה עכשיו דרך גוגל במקרה
הגעתי דרך
תמיד בדוק בערך האנגלי אם יש מפה או תמונה
האנגלי בדוק
בינתיים המפעילים בחדר הבקרה הח

מכיוון שזו אנציקלופדיה אני רואה בערך כזה ערך בשלבי כתיבה
שזו זו
את מחפשת פורום מחשבים בתפוז
מחפשת פורום
נובגורוד נחשבת לעיר החשובה ביותר מבחינה היסטורית בצפון מערב רוסיה
רוסיה נובגורוד
אני מקווה שעוד חברים יקראו את חטיבת השיחה הנוכחית
יקראו אני
אפשר למחוק השאר יבוצע בהמשך
יבוצע שאר
לכן האופציה היחידה למחיקה היא להבנתי הצבעת מחיקה
להבנתי אופציה
לגבי זמנו הפנוי זה לקוח מהסרט שמופיע באתר שלו
הפנוי זה
ככה אנו רוצים לנהל את העסק
העסק אנו
אך סטנלי לא מוכן ללכת ללא אפס
סטנלי סטנלי
1
ואם יש עדיין בעיות עריכה אז תודיע לי
תודיע בעיות
רק חבל שאתה נושא בתפקיד בירוקרט בויקיפדיה
בירוקרט חבל
אינני יודע גם אם יש עוד בתי מדרש לרבנים
לרבנים בתי
אבל צעד צעד עזרה בהערות כלליות לא עזרו עד כה
כלליות צעד
הדגל חביב אין לי התנגדות לו
חביב דגל
שיש בדיוק גרסה אחת להפניה
להפניה גרסה
הוא קיבל גם את הכינוי גורדון הסיני
גורדון הוא
אין ספק שהוא איש חשוב בחולון העושה רבות למען העיר
העושה ספק
חברים לא חסרים לנו רעיונות טובים
רעיונות חברים
התוכן נוצר אוטומטית כאשר יש מספיק פרקים
נוצר תוכן
אם התופעה קיימת אבל ההגדרות חדש

אם לא יהיו מקורות מהימנים תוך פרק זמן סביר אמחק
מהימנים מקורות
על כולנו מקובל שאין להם חופש לכתוב בויקיפדיה
כולנו חופש
למרות זאת השפעתו על סצנת הרוק המתקדם באיטליה הייתה גדולה
הרוק השפעתו
שוב אתה נוהג כאילו דעתך עליונה כאן ולא כך הדבר
עליונה דעתך
הוא מאמר קצת יותר טוב אבל רק קצת
מאמר הוא
אם זה לא דיון ויקיפדי אינני יודע מה כן
ויקיפדי זה
ותמונה אחת לפחות חייבים גם
ותמונה תמונה
1
אם הורדת את ההערה נדמה לי שצריך למחוק גם אותן
הורדת הורדת
1
עם נפילת שושלת טאנג מסתיימת תקופת סין הקיסרית המוקדמת
שושלת תקופת
אתה מוזמן לתרגם לעברית קלה את משוואות מקסוול
משוואות אתה
להתפתחות הרוק הברזילאי חשיבות רבה גם בהיסטוריה של הרוק בעולם
להתפתחות חשיבות
שלום אסף ב וברוך בואך לוויקיפדיה האנציקלופדיה העברית החופשית הראשונה
בואך שלום
יש משהו פנימי בתוך קרן ויקימדיה
קרן משהו
ביס יסודיים באחריות העיריה ממלכתי דתי
יסודיים ביס
שניהם חייבים לעשות זאת אך מסיבות שונות
שניהם שניהם
1
יום טוב פורומים ואתרי מעריצים לא נהוג לקשר אליהם
ואתרי יום
תמיד יש תחום אפור זו לא המדיניות אלא השוליים שלה
השוליים תחום
מצחיק אני ביקשת

...

לדעתי אין שום פגם בערך הזה הוא פשוט מושלם
מושלם פגם
ערך זה עוסק בעיר ורסאי סמוך לפריז
לפריז ערך
האם אנחנו יכולים לרוץ כמו נמר
נמר אנחנו
הוא משחק כדורסל פשוט יוצא מן הכלל
כדורסל הוא
זו לא צריכה להיות בעיה כזו גדולה
כזו זו
תמונות או איורים נוספים ימחקו אוטומטית
איורים או
האם מישהו יכול לשפוך אור על נקודה חשובה זו
נקודה מישהו
הדיון ככלל קצת סוער מדי קטגוריה אינה אלא עזר ניווט
סוער דיון
גאוגרפיה מפת ג מייקה פרק זה לוקה בחסר
ג גאוגרפיה
1
סרטי גמר של יוצרים צעירים רבים מבוססים על סיפוריו
סיפוריו סרטי
בדיקות על תבנית זו נמצאות ב מפת מיקום קובה בדיקות
קובה בדיקות
שם זה מרמז על דמיון הפרח לדו חי
דמיון זה
קישורים חיצוניים באתר הבמה אתר האינטרנט
הבמה קישורים
מבחינתי הערך גמור ואף הסרתי את התבנית
הסרתי ערך
העיר אלכסנדריה הורחבה אף היא
אלכסנדריה עיר
אינני יודע מדוע אבן שושן בחר ב מירוץ
שושן יודע
דני נוימן עובד מאז פרישתו בתחום התקשורת
דני דני
1
לא כל מה שהם מציעים הוא שינוי רדיקלי
רדיקלי מה
הבעיה קשה לשכנע למחוק משהו קיים
לשכנע בעיה
הוא קשור בצורה כלשהי לערך גישת עיבוד המידע
גישת הוא
כנראה שאנחנו ל

לפני ואחרי מקף מפריד יש תמיד רווח
מקף רווח
השניים הופכים להיות חברים קרובים
קרובים שניים
אגב לא הייתה שום גרסה יציבה לערך
יציבה גרסה
לא פעם אף נמחק הדף כולו ללא הפנייה וללא התראה
התראה דף
אני לא מצליח להבין את המושגים המקצועיים
המקצועיים אני
מנכל החברה הנוכחי הוא ג ורג דיוויד
ג מנכל
ספרים ו לך חפש מאיפה הטענה הזו הגיעה
חפש טענה
אם אתה יודע שזה טעות אתה יכול למחוק
טעות אתה
בערך העברי של הביטלס אין אפילו תמונה אחת לרפואה
לרפואה תמונה
אתה מציע כאן פרוייקט יפה אבל זה פשוט לא המקום
אתה אתה
1
ויקיפדיה מוצג טוב בפיירפוקס וייתכן וקיימת בעיה כלשהי בדפדפן שלך
וקיימת בעיה
הם נראים לי בסדר גמור אבל הקטגוריות חסרות מאוד
נראים הם
אני אישית בעד האופציה השנייה
האופציה אני
יובל מדר תגובה מנומקת ומה רע בקטגוריה נוספת
מנומקת יובל
מספיק הבהרת חשיבות והערך ימחק תוך יום
הבהרת ערך
לא כל מה שקיים בויקיפדיה האנגלית פסול
שקיים מה
המבנה עצמו היה לבית אירוח של בית סבויה
סבויה מבנה
למעשה אלו שתי האופציות בויכוח עם נזקק אותו
נזקק אלו
צבעי הדגל הם צבעי דגל סקוטלנד
סקוטלנד צבעי
אם אתה רוצה לחסום אותי בקש זאת
בקש אתה


היום זה קמר מחר זה אצלי צ כלברה
צ זה
פוסט פאנק התפתח בסוף שנות השבעים אל תוך השמונים
פאנק פוסט
שימוש נרחב בסמים הסמים היוו גורם דומיננטי בקרב הקהל
דומיננטי שימוש
האם אתה מתיר שימוש חופשי בהם
מתיר אתה
באיזה מובן שרד העם היהודי ביחס לאומות אחרות
לאומות עם
צריכה להיות מחאה עממית מכל שכבות הציבור
עממית מחאה
הממלכה המאוחדת שוכנת בין צפון האוקיינוס האטלנטי לים הצפוני
האטלנטי ממלכה
היא גם לא תסכים להפסיד שבועיים עבודה
להפסיד היא
התחרות בין צורות התחבורה השנה סייעה בשמירה על מחירים נמוכים
בשמירה תחרות
אני מזמין אותך בעוד חודש לדון בנושא בדף השיחה
אני אני
1
הערך צריך לעבור לנוהל טיפול חירום
חירום ערך
אבל דומני שאין חילוקי דיעות כאן
דיעות דומני
אלה רק חלק מן הפרסים הרבים בהם זכתה במהלך השנים
הפרסים אלה
גם מקור המשמעות השנייה ב חול
המשמעות מקור
בנו עומר פרץ הוא כדורגלן המשחק בקבוצת מכבי תל אביב
כדורגלן בנו
את יכולה למחוק אותה אבל שאר התמונות הן חוקיות לגמרי
חוקיות הן
בטיפול שלי אני מבקש הנחה של חצי מחיר לפחות
בטיפול אני
אני פחות מבין במתמטיקה ממך זה בטוח
במתמטיקה אני
אם אתה צריך בעתיד עזרה בעניינ

אבל מעבר לזה אני לא חושב שזה פועל ככה
פועל אני
תחנת רדיו חוקית ישראלית לא זכאית לערך
רדיו תחנת
שלחתי לך דואר אלקטרוני באמצעות מערכת ויקיפדיה
דואר שלחתי
1
הוא היה חובב ספורט בצעירותו ושיחק כדורגל בקבוצה פולנית
פולנית הוא
ג לגבי המדיניות של ארהב זה נתון לויכוח
לויכוח זה
לטענות אלו אין ביסוס עובדתי
לטענות ביסוס
ארגוב ראוי לערך רציני יותר כולל דיסקוגרפיה מלאה וכו
דיסקוגרפיה ארגוב
גם זה דומה לערך שלנו בגרסת באופן חשוד
בגרסת זה
הוא היה בין מייסדי חוג שבעים ושבע
שבעים הוא
אולם לצערי ויקיפדיה אינה אתר הנצחה
הנצחה ויקיפדיה
בעקבות הביקוש הרב נוספו שתי הקרנות נוספות לסרט
הקרנות הקרנות
1
טקסט בערך צריך להיות זורם העין צריכה לרוץ לאורכו
זורם טקסט
במקום שבו יש פנייה בלשון זכר יש לתקן את הניסוח
פנייה פנייה
1
אני גם לא טוען שזה ברווז עיתונאי
ברווז אני
השאלה היא איזו קהילה אנו רוצים בעתיד
בעתיד שאלה
מצווה זו היא המצווה הראשונה המופיעה בתורה
המופיעה מצווה
במלחמת העצמאות שימש בסיס להתקפות ערביות על הדרך לירושלים
ערביות בסיס
בו בזמן הסיפור מראה פחד יסודי אצל היהדות הרבנית
הרבנית פחד
איזה זרם מחשבה הינו נפ

אני שב ומציע למחוק את כל התמונות עם תבנית זו
ומציע אני
המשחקים נערכו ברובם במתחם האולימפי שבעיר בייג ינג בירת סין
בייג משחקים
מאידך יתכן וזה בז עצים צעיר
מאידך זה
כל אחד יוצר שדה המכיל את הפונקציות האפשריות עבור עצמו
האפשריות אחד
אני לא חושב שמישהו צייר את זה קודם
צייר אני
ארצות הברית הייתה בדרכה לשיא כוחה הצבאי
לשיא ארצות
קישורים חיצוניים האתר הרשמי של מושל ג ורג יה
ג קישורים
אליס מילר התקבלה לקורס טיס אך הודחה עקב עודף מוטיבציה
אליס אליס
1
הספר הינו תיאור רב עוצמה ועדות אישית המהווה הישג ספרותי
ספרותי ספר
1
אתה מציע למחוק את הערכים
אתה אתה
1
אני מבקש למחוק את המשפט
אני אני
1
השינויים שבוצעו בערך אינם שלי כי אם של הקהילה
שבוצעו שינויים
רוב הנוסעים היו ניצולי שואה
ניצולי נוסעים
בכל אופן אין סיבה למחוק את דף המשתמש של נמרוד
נמרוד סיבה
ראיתי שגם פרס ברנר טרם הועלה
ברנר פרס
יש ממשל צבאי שהוקם ברוח אמנת ז נבה
אמנת ממשל
עדיין לא הבנתי איזה כלל תומאס הפר
הפר פר
1
בשני הקרבות הובס צבא הקונפדרציה
הובס צבא
אל תל חי הגיעו קומץ מתנדבים בפיקודו של יוסף טרומפלדור
בפיקודו קומץ
הייתי עושה זאת בעצמי א

In [77]:
real_count,counter,new_counter

(1383, 0, 0)

In [78]:
real_count

1383